<a href="https://colab.research.google.com/github/aisaza609/Datos_a_la_U_grupo_2/blob/main/Normalizacion_Dataset_Neiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import io
from io import BytesIO
import random
import ipywidgets as widgets
from IPython.display import display
!pip install gdown
import gdown
%matplotlib inline

In [4]:
# Definimos el ID del archivo de Google Drive y generamos la URL de descarga
file_id = '1pHF94shp0VZPkBXD4QKZM1XnLq31COL5'  # Reemplaza esto con el ID del archivo en Google Drive
url = f'https://drive.google.com/uc?id={file_id}'

# Descargamos el archivo y lo guardamos con el nombre df_neiva.csv
output = 'df_neiva.csv'
gdown.download(url, output, quiet=False)

# Cargamos el archivo en un DataFrame de pandas llamado df_neiva para empezar a trabajar con los datos
df_neiva = pd.read_csv('df_neiva.csv')
print(df_neiva.head())  # Verificamos las primeras filas para asegurarnos de que se cargó correctamente

Downloading...
From: https://drive.google.com/uc?id=1pHF94shp0VZPkBXD4QKZM1XnLq31COL5
To: /content/df_neiva.csv
100%|██████████| 4.24M/4.24M [00:00<00:00, 24.7MB/s]


  TIPO        FECHA_NACIMIENTO  MUNICIPIO SEXO COD_DX  \
0   CC  09/24/1972 12:00:00 AM     ITAGUI    F   D172   
1   CC  01/29/1981 12:00:00 AM  SAN PEDRO    M   S523   
2   CC  09/11/1973 12:00:00 AM     ITAGUI    F   S324   
3   RC  04/07/2018 12:00:00 AM   ENVIGADO    F   J042   
4   CC  01/03/1994 12:00:00 AM   MEDELLIN    F   S823   

                                           NOMBRE_DX  \
0  TUMOR BENIGNO LIPOMATOSO DE PIEL Y DE TEJIDO S...   
1                  FRACTURA DE LA DIAFISIS DEL RADIO   
2                             FRACTURA DEL ACETABULO   
3                            LARINGOTRAQUEITIS AGUDA   
4       FRACTURA DE LA EPIFISIS INFERIOR DE LA TIBIA   

                                            ENTIDAD  
0                                      PARTICULARES  
1                    SEGUROS GENERALES SURAMERICANA  
2  ADRES (ADMINISTRADORA DE LOS RECURSOS DEL SGSSS)  
3                        EPS SURA CONTRIBUTIVO 2020  
4                    ALIANZA MEDELLIN ANTIOQUIA SA